# Bedrock Knowledge Base Retrieval and Generation with Guardrails

## Description
This notebook demonstrates how to enhance a Retrieval-Augmented Generation (RAG) pipeline using Amazon Bedrock with Guardrails for better model control and filtering. We will walk through retrieving data from a knowledge base, applying guardrails to control the generation of responses, and filtering results using metadata.

![Guardrails](./guardrail.png)

## 1: Import and Load Variables

In [ ]:
import json

# Load the configuration variables from a JSON file
with open("../Lab 1/variables.json", "r") as f:
    variables = json.load(f)

variables


## 2: Define ARN and Configuration Details

In [ ]:
# Setting up configuration for Bedrock and Guardrails
accountNumber = variables['accountNumber']
knowledge_base_id = variables['kbFixedChunk']   
model_id = 'us.amazon.nova-pro-v1:0' 
guardrail_version = variables['guardrail_version'] 
guardrail_id = variables['guardrail_id']    

# Define ARNs (Amazon Resource Names) for the model and guardrails
model_arn = f"arn:aws:bedrock:us-west-2:{accountNumber}:inference-profile/{model_id}"
guardrail_arn = f'arn:aws:bedrock:us-west-2:{accountNumber}:guardrail/{guardrail_id}'  # Replace with your guardrail ARN


## 3: Set Up Bedrock Client

In [ ]:
import boto3

# Configure the Bedrock client
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name="us-west-2")


## 4: Define Function for Retrieval with Guardrails

In [ ]:
def retrieve_and_generate_with_conditional_guardrails(
    query, 
    knowledge_base_id, 
    model_arn, 
    metadata_filter=None,
    use_guardrails=False,
    guardrail_id=None,
    guardrail_version=None
):
    """
    Retrieves and generates a response with optional Guardrails application.
    
    Parameters:
    - query (str): The input query.
    - knowledge_base_id (str): The ID of the knowledge base.
    - model_arn (str): The ARN of the model.
    - metadata_filter (dict, optional): The filter for the vector search configuration.
    - use_guardrails (bool, optional): Whether to apply guardrails. Defaults to False.
    - guardrail_id (str, optional): The ID of the guardrail to apply. Required if use_guardrails is True.
    - guardrail_version (str, optional): The version of the guardrail. Required if use_guardrails is True.
    
    Returns:
    - response: The response from the retrieve_and_generate method.
    """
    # Start with base configuration
    kb_config = {
        'knowledgeBaseId': knowledge_base_id,
        "modelArn": model_arn,
        "retrievalConfiguration": {
            "vectorSearchConfiguration": {
                "numberOfResults": 5
            }
        }
    }
    
    # Add metadata filter if provided
    if metadata_filter:
        kb_config["retrievalConfiguration"]["vectorSearchConfiguration"]["filter"] = metadata_filter
    
    # Add generation configuration with prompt template
    kb_config["generationConfiguration"] = {
        "promptTemplate": {
            "textPromptTemplate": "Answer the following question based on the context:\n$search_results$\n\nQuestion: {question}"
        }
    }
    
    # Add guardrail configuration only if requested
    if use_guardrails:
        # Validate required parameters
        if not guardrail_id:
            raise ValueError("guardrail_id is required when use_guardrails is True")
        
        guardrail_config = {
            "guardrailId": guardrail_id
        }
        
        # Add version if provided
        if guardrail_version:
            guardrail_config["guardrailVersion"] = guardrail_version
            
        # Add to generation configuration
        kb_config["generationConfiguration"]["guardrailConfiguration"] = guardrail_config
    
    # Make the API call
    response = bedrock_agent_runtime.retrieve_and_generate(
        input={
            "text": query
        },
        retrieveAndGenerateConfiguration={
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": kb_config
        }
    )
    
    return response

## 5: Define Metadata Filter

In [ ]:
# Define a metadata filter for advanced filtering based on specific conditions
one_group_filter= {
    "andAll": [
        {
            "equals": {
                "key": "docType",
                "value": '10K Report'
            }
        },
        {
            "equals": {
                "key": "year",
                "value": 2023
            }
        }
    ]
}


## 6: lets validate if the guardrails restrict any investment advice.
lets ask the Foundational model for an investment advice. When we created the guardrails, we restricted bedrock to provide any investment advice. Bedrock should be return a  preconfigured response "This request cannot be processed due to  safety protocols"

In [ ]:
# Define the query that will be sent to the model
query = "based on your amazon's results should I buy amazon stock?"


In [ ]:
response_without_guardrails = retrieve_and_generate_with_conditional_guardrails(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn,
    metadata_filter=one_group_filter,
    use_guardrails=False  # Explicitly set to False, 
)

print(response_without_guardrails['output']['text'])

## 7: Retrieve Response with Guardrails

In [ ]:
response_with_guardrails = retrieve_and_generate_with_conditional_guardrails(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn,
    metadata_filter=one_group_filter,
    use_guardrails=True,
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version
)
print(response_with_guardrails['output']['text'])

## 8. Guardrails for PII data. 

In [ ]:
query="Who is the current CFO of Amazon?"


In [ ]:
response_without_guardrails = retrieve_and_generate_with_conditional_guardrails(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn,
    metadata_filter=one_group_filter,
    use_guardrails=False  # Explicitly set to False, 
)

print(response_without_guardrails['output']['text'])

In [ ]:
# response_with_guardrails=retrieve_and_generate_with_guardrails(query, knowledge_base_id, model_arn,guardrail_id,guardrail_version,one_group_filter)
# print(response_with_guardrails['output']['text'])   
response_with_guardrails = retrieve_and_generate_with_conditional_guardrails(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn,
    metadata_filter=one_group_filter,
    use_guardrails=True,
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version
)
print(response_with_guardrails['output']['text'])